In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm, ncx2, chi2
#plt.style.use("dark_background")

# Tutorial and toy comparison between different statistical tests of detection

Measurement instruments, (astronomical instruments included) produce obsevable quantities (i.e. numbers) designed to inform us on the object of our inquiry. However, measurements are random variables, meaning that their values vary randomly around the actual measure according to a statistical law. In other words, the measurement process produce some "noise" around the "signal" that we want to study. While some conclusions at high S/N (signal to noise ratio) can be made at first glance when looking at the data, most cutting-edge results can only be made at low S/N. In this regime, it can be difficult to make (statistically) informed decisions based on the wiggling numbers, especially since it we are not very familiar with the values we expect in the presence of significant signal.

This notebook will guide you through some of the basic mathematical tools necessary to do a practical implementation of the different tests showcased in **Ceau et al. 2019**$^1$. It will guide you from basic concepts of statistics all the way to powerful, field-ready statistical tests. The formalism described here is not the only one that is mathematically sound, and many bridges can be found to other approaches.

$^1$ https://ui.adsabs.harvard.edu/abs/2019A%26A...630A.120C/abstract

## Prerequisites on errors

Here we are mostly interested in the detection of features around point sources, in other words, we wish to be able to chose between **two mutually exclusive scenarios** (called hypotheses).
The principle of basic statistical tests is to assess the risk of being wrong when rejecting one hypothesis, called the null hypothesis. 

In our case, the **null hypothesis, $\mathcal{H}_0$**, always represents the default case where there is no signal of interest, that is of a perfectly point-like source (more precisely a perfectly centro-symmetric souce in the case of **closure phase**, **kernel phase** or **kernel null** data, including the **differential null** obtained with double-Bracewell nullers).

The **alternative hypothesis $\mathcal{H}_1$** represent the case where a signal of interest is present in the meausurement.

To be able to calculate the risk of rejecting $\mathcal{H}_0$, we need to model the measurement process (the model must be valid in both hypothesese).
In our case, we will use the following:

We consider that the measurments described by the vector $\mathbf{y}$ gathering a series of $n$ values and are affected by a random error described by the vector $\boldsymbol{\varepsilon}$ and a signal of interest represented by the model vector $\mathbf{x}$.

As mentionned above, under $\mathcal{H}_0$, $\mathbf{x}$ is a null vector and we can write:

$$
\begin{cases}
    \mathcal{H}_0 : \mathbf{y} = \boldsymbol{\varepsilon} \\
    \mathcal{H}_1 : \mathbf{y} = \mathbf{x} + \boldsymbol{\varepsilon}
\end{cases}
$$



This kind of model always assume that the error vector $\boldsymbol{\varepsilon}$ follows :
1. a normal distribution 
1. of stadard deviation 1
1. centered around 0
1. with uncorrelated elements

In summary $ \boldsymbol{\varepsilon} \sim \mathcal{N}(0,\mathbf{I})$

In this context, 1. is generelly true, and 3. is true in theory. However in most cases 2. and 4. are generally false with interferometric data$^2$. However, simple algebraic procedures can be used to translate the raw data $\mathbf{y}'$ into usable observables $\mathbf{y}$ that fit these prerequesites. **Those will be described in a different tutorial**, but a preview is showed in appendix A.

Note that in practice, the true limitation of this testing approach are determined by our ability to:
* calibrate (and therefore measure) the biases and therefore enforce 3,
* evaluate the amplitude of errors and evaluate and therefore enforce 2.


$^2$ : In the case of Closure/Kernel phases, the pixel-level (photon and read) noises become correlated. Even in the case of kernel-nulls where the observable quantity comes from the simple difference of two pixels, the instrumental errors are correlated between the different wavelength channels.


## First create some model ($\mathbf{x}$) and data ($\mathbf{y}$).

$\mathbf{x}$ is a vector corresponding to the signal we want to detect. It is the expectation of $\mathbf{y}$ under $\mathcal{H}_1$.

Here we use a simple sine wave. Adjust the amplitude `A` to create different SNR. (We will leave the standard deviation of noise to 1.)

In [ ]:
A = 0.85 # default = 0.85
ndim = 50 # default = 50
phi = 0.2 # default = 0.2
t = np.arange(ndim, dtype=np.float64)
x = A*np.sin(0.5*t + phi) # x is the model signal

In [ ]:
# Here we create the measurement noise epsilon around the signal x
noise_bank = np.random.multivariate_normal(mean=np.zeros(ndim),cov=np.eye(ndim), size=1000)


$\mathbf{y}$ is a vector of observable quantities obtained in an experiment. For the sake of this demonstration, we will draw two versions:
* `y_0` obtained under $\mathcal{H}_0$ (so just noise),
* and `y_1` obtained under $\mathcal{H}_1$, (so the noise offset by a value corresponding to the model `x`).


In [ ]:
y_0 = noise_bank[0,:]
y_1 = x + noise_bank[1,:]

In [ ]:
# Let's have a look at the data we've generated!
fig = plt.figure(dpi=150)
plt.plot(y_0, label=f"$y$ (obsered under $\mathcal{{H}}_0$, just noise)", color="C0")
plt.plot(y_1, label=f"$y$ (observed under $\mathcal{{H}}_1$, noisy signal)", color="C1")
plt.plot(x, label=f"$x$ (model signal)", color="C1", linestyle="--")
plt.legend(fontsize="xx-small")
plt.show()

### Fig. 1:  A plot of the model signal, the noise, and the noisy signal. With the default parameters, the signal under $\mathcal{H}_1$ is "buried" in the noise.

As we can see in Figs. 1 and 2, we are in a case of low S/N, in which it is difficult to give an interpretation by eye. This is a case where statistics must be used to inform on a result, both for the instrument scientist trying to predict the performance of the instrument, and for the observer, trying to decide what conclusion to draw.

This is called statistical inference. 


In [ ]:
xvals = np.linspace(np.min(x), np.max(x), 100)
fig = plt.figure(dpi=150, figsize=(6,3))
plt.subplot(121)
plt.errorbar(x, y_0, yerr=1., fmt="none", label=f"$\mathbf{{y}} | \mathcal{{H}}_0$", 
            alpha=0.5)
plt.plot(xvals, xvals, color="gray", linestyle="--")
plt.legend(loc="lower right")
plt.ylabel(f"Observation $y$")
plt.xlabel(f"Model $x$")
plt.subplot(122)
plt.errorbar(x, y_1, yerr=1., fmt="none", label=f"$\mathbf{{y}} | \mathcal{{H}}_1$",
            alpha=0.5)
plt.plot(xvals, xvals, color="gray", linestyle="--")
#plt.gca().set_aspect("equal")
plt.xlabel(f"Model $x$")
plt.legend(loc="lower right")
#plt.tight_layout()
plt.show()

### Fig. 2:  A correlation plot under $\mathcal{H}_0$ and $\mathcal{H}_1$. This type of plots help to identify the resemblance between the model and the data, by plotting each on a different axis, with a dashed line representing $\mathbf{y} = \mathbf{x}$ . In cases with high S/N, we sould be able to see the correlation trend between the two. In cases that are actually a challenge for the detection test, it should be barely noticeable.

# Statistical tests

Statistical test or hypothesis tests are built around the statistics of the errors in the data. They provide a decision tool that is built around the false alarm rate $P_{FA}$  which is the probability to reject $\mathcal{H}_0$ when it is actually true.

The approach we follow here will be driven by the choice of a value for $P_{FA}$. 

### You can adjust your False Alarm rate $P_{FA}$ here.

In [ ]:
Pfa = 0.05 # default: 0.05

A test is composed of two main ingredients:
* A test statistic $T: \mathbf{y} \rightarrow T(\mathbf{y})$ that can be computed based on observed data.
* A threshold $\xi$ that will be used to select an hypothesis or make a decision.

The way that $\xi$ can be computed to obtain certain properties (most notably a certain $P_{FA}$) is one of most important aspects of a detection test.

Depending on how $T(\mathbf{y})$ compares with $\xi$, we will accept either one or the other of the hypotheses.

$$T(\mathbf{y}) \underset{\mathcal{H}_0}{\overset{\mathcal{H}_1}{\gtrless}} \xi $$


In the approach taken by Ceau et al. 2019, $T$ is constructed based on a likelihood ratio
$$ \frac{\mathscr{l}(\mathcal{H}_1; \mathbf{y})}{\mathscr{l}(\mathcal{H}_0; \mathbf{y})}  \underset{\mathcal{H}_0}{\overset{\mathcal{H}_1}{\gtrless}} \xi' $$
This likelihood can be expressed (**Scharf & Friedlander 1994**):
$$ \mathscr{l}(\mathcal{H}_1; y) =  (2\pi)^{-\frac{n}{2}} \mathrm{exp}\Big( -\frac{1}{2}(\mathbf{x} - \mathbf{y})^T(\mathbf{x} - \mathbf{y}) \Big)$$
where $n$ is the length of the data vectors. By simplifying and taking the logarithm, this ratio can be simplified and turned into a difference.

The main difference between the different tests being how well refined the model is under $\mathcal{H}_1$, leading to different expressions of $\mathscr{l}(\mathcal{H}_1; y)$.

# Energy detector test

Under this test, nothing is known on the type of signal under $\mathcal{H}_1$. Under such circumstances, the best estimate for the model is $\hat{\mathbf{x}} = \mathbf{y} $, leading to equation (21) in **Ceau et al. 2019**, and therefore:
$$ T_E(\mathbf{y}) := \mathbf{y}^T\mathbf{y} \underset{\mathcal{H}_0}{\overset{\mathcal{H}_1}{\gtrless}} \xi $$

This is extremely simple. It is also referred to as the $\chi^2$ test. It is adequate when nothing is known about the signature $\mathbf{x}$.

But as you may have guessed, a test is worthless without a rational way to set its threshold $\xi$. This can be expressed with based on the distribution of $T_E$ under the different hypotheses:
$$
\begin{cases}
    \mathcal{H}_0 : T_E(\mathbf{y}) \sim \chi^2_p (\mathrm{loc}=0) \\
    \mathcal{H}_1 : T_E(\mathbf{y}) \sim \chi^2_p (\mathrm{loc}=\mathbf{x}^T\mathbf{x})
\end{cases}
$$
Note how therefore be expressed based on the inverse of the Cumulative Distribution Function ($\mathrm{CDF}^{-1}$) of the $\chi^2$ distribution. This inverse is somtimes called Percent Point Function ($\mathrm{PPF}$ and we will find it under this name in `scipy.stats.ncx2`, here imported as `ncx2`.$^2$ Pay close attention to the way those parameters are provided to the function, as this api is relatively tricky. The following cell queries the first lines of the documentation string of this class:

$^2$: Note that we could also have used `scipy.stats.chi2` for the cases under $\mathcal{H}_0$, but here we prefer the uniformity.

In [ ]:
print(ncx2.__doc__[:1100], "...")

In [ ]:
xsi = ncx2.ppf(1-Pfa, df=ndim, nc=0.)
print(f"Pfa (chosen) = {Pfa}")
print(f"xsi = {xsi}")

Now determining $P_{Det}$ requires assumptions on the signal to determine how its amplitude is translated into values of $T_E$. Here, we use the model signal $\mathbf{x}$ that we prepared before.

In [ ]:
xTx = x.T.dot(x)
Pdet_Pfa = 1 - ncx2.cdf(xsi, ndim, xTx)
print(f"Pfa (chosen) = {Pfa}")
print(f"Pdet = {Pdet_Pfa}")

This is best represented as PDF plots:

In [ ]:
z = np.linspace(0., 2*xsi, 1000)
zdet = z[z>xsi]
zndet = z[z<xsi]
fig = plt.figure(dpi=150)
plt.plot(z, ncx2.pdf(z, df=ndim, nc=0), label=f"PDF($T_{{E}} | \mathcal{{H}}_0$)")
plt.fill_between(zdet,  ncx2.pdf(zdet, df=ndim, nc=0), alpha=0.3, label=f"$P_{{FA}}$")# , hatch="//"
plt.plot(z, ncx2.pdf(z, df=ndim, nc=xTx), label=f"PDF($T_{{E}}| \mathcal{{H}}_1$)")
plt.fill_between(zdet,  ncx2.pdf(zdet, df=ndim, nc=xTx), alpha=0.3, label=f"$P_{{Det}}$")
plt.axvline(xsi, color="gray", linestyle="--", label=f"$\\xi(P_{{FA}}={Pfa})$")
plt.xlabel(f"$T_{{E}}$")
plt.ylabel(f"$PDF(T_{{E}})$")
plt.legend()
plt.show()

### Fig. 3 : The PDF (probability density function) of $T_E(\mathbf{y})$ under both hypotheses. We cans see $\xi$ as the user's attemps to discriminate between the two hypotheses in a clean cut. The integrals represented by filled regions correspond to the values of $P_{FA}$ and $P_{det}$. With the default parameters, this not very sensitive and rejects most of the cases under $\mathcal{H}_1$.

The test therefore tells us if we have statistical reasons to reject the Null hypothesis ($\mathcal{G}_0$, that is that the target has no detectable features). Gaining knowledge on what features must be done by other means (model fitting, statistical inference, image reconstruction...).

Note that this energy detector test is not very powerful. It is however still usefull, as $P_{Det}$ and the sensitivity can be evaluated mathematically, and so it can help determine the a lower bound for the performance of an instrument.

# Neyman-Pearson test

Such a likelihood ratio test will be most sensitive when we perfectly know the expected signature. In this case, the test statistic can be written:
$$ T_{NP}(\mathbf{y}, \mathbf{x}) = \mathbf{y}^T\mathbf{x} \underset{\mathcal{H}_0}{\overset{\mathcal{H}_1}{\gtrless}} \xi $$
(see equations (13) and (14) of **Ceau et al. 2019**.


$$
\begin{cases}
    \mathcal{H}_0 : T_{NP}(\mathbf{y}, \mathbf{x}) \sim \mathcal{N} (\mathrm{loc}=0, \mathrm{scale}=\sqrt{\mathbf{x}^T\mathbf{x}}) \\
    \mathcal{H}_1 : T_{NP}(\mathbf{y}, \mathbf{x}) \sim \mathcal{N} (\mathrm{loc}=\mathbf{x}^T\mathbf{x}, \mathrm{scale}=\sqrt{\mathbf{x}^T\mathbf{x}}) 
\end{cases}
$$

Note how in this case, the test statistics (and its distribution) also depend on a model signal $\mathbf{x}$.

This time, we will need a model for the normal distribution that we find in `scipy.stats.norm` here imported as `norm`. We lookup the api for it with the next cell:

In [ ]:
print(norm.__doc__[:1100], "...")

In [ ]:
xTx = x.T.dot(x)
xsi = np.sqrt(xTx)*norm.ppf(1-Pfa)
print(f"xsi = {xsi}")

In [ ]:
Pdet_Pfa = 1 - norm.cdf((xsi - xTx)/np.sqrt(xTx), loc=0, scale=1)
print(f"Pfa (chosen) = {Pfa}")
print(f"Pdet = {Pdet_Pfa}")

We can see that (with the default parameters) the detection is still very high. This shows that this test is more sensitive than the energy detector ($T_E$) test.

In [ ]:
z = np.linspace(-0.5*xTx, 4*xsi, 1000)
zdet = z[z>xsi]
zndet = z[z<xsi]
fig = plt.figure(dpi=150)
plt.plot(z, norm.pdf(z, loc=0, scale=np.sqrt(xTx)), label=f"Pdf($T_{{NP}} | \mathcal{{H}}_0$)")
plt.fill_between(zdet,  norm.pdf(zdet, loc=0, scale=np.sqrt(xTx)), alpha=0.3, label=f"$P_{{FA}}$")# , hatch="//"
#plt.fill_between(z[], )
plt.plot(z, norm.pdf(z, loc=xTx, scale=np.sqrt(xTx)), label=f"Pdf($T_{{NP}}| \mathcal{{H}}_1$)")
plt.fill_between(zdet,  norm.pdf(zdet, loc=xTx, scale=np.sqrt(xTx)), alpha=0.3, label=f"$P_{{Det}}$")
plt.axvline(xsi, color="gray", linestyle="--",  label=f"$\\xi(P_{{FA}}={Pfa})$")
plt.xlabel(f"$T_{{NP}}$")
plt.ylabel(f"$PDF(T_{{NP}})$")
plt.legend()
plt.show()

### Fig. 4 : The PDF for $T_{NP}(\mathbf{y})$, to be compared with Fig.3. Here, the result is more sensitive than with $T_E$ and under the default parameters, it catches most of the cases under $\mathcal{H}_1$ (i.e. $P_{det}$ is high).

Note that this Neyman-Pearson test is the most powerfull test possible. However, it is also much less useful in practice. Indeed, it requires that the signature sought be known in advance. Therefore in can only be used to confirm a detection, but the result will bring no additional knowledge on the feature detected.

However, it allows us to compute relatively easily the theoretical upper bound of the performance in detection of a given instrument or approach.

# Receiver operator characteristic

The performance of the tests can be compared as a function of chosen false alarm probability $P_{FA}$. This is called the Receiver Operator Characteristic (ROC) plot for a given model signal $\mathbf{x}$.

The diagonal of the plot ($P_{Det}=P_{FA}$) represents a purely random test, with a coin toss located at $(0.5, 0.5)$. Note that we are usually interested in the performance at very low false alarm rates.

In [ ]:
#Pfas = np.linspace(0., 0.1, 1000)
Pfas = np.logspace(-10,-0.2, 100)
Pdet_NP_Pfa = 1 - norm.cdf((np.sqrt(xTx)*norm.ppf(1-Pfas) - xTx)/np.sqrt(xTx))
Pdet_E_Pfa = 1 - ncx2.cdf(ncx2.ppf(1-Pfas, ndim, 0.), ndim, xTx)
fig = plt.figure(dpi=150)
plt.plot(Pfas, Pdet_NP_Pfa, label=f"ROC($T_{{NP}}$)")
plt.plot(Pfas, Pdet_E_Pfa, label=f"ROC($T_{{E}}$)")
plt.plot(Pfas, Pfas, color="gray", linestyle="--", label="Random")
plt.axvline(Pfa, color="gray", label=f"$P_{{FA}}={Pfa:.3f}$")
#plt.gca().set_aspect("equal")
#plt.xscale("log")
#plt.yscale("log")
plt.xlabel(f"$P_{{FA}}(\\xi)$")
plt.ylabel(f"$P_{{Det}}(\\xi)$")
plt.ylim(0, 1)
plt.xlim(0, None)
plt.legend()
plt.show()


### Fig. 5: The compared ROC for $T_{NP}$ and $T_E$ for the model signature $\mathbf{x}$. We can see the $T_{NP}$ is extremely sensitive even at low $P_{FA}$.

# Matched subspace detector

Tests can be found in between those two limit performances that are adapted to the detection of specific families of signatures, rather than one exact signature. This constitutes a *Generalized* Likelihood Ratio test (GLRt). For this we build a family of signals $\{\mathbf{x}\}=\{f(\mathbf{a})\}$ that are spawned from a model of a small number of parameters here represented by the vector $\mathbf{a}$ such that it represents a subspace of the possible models. For the example of a companion fainter star these parameters could be $\rho$ the separation, $\theta$ the position angle, and $c$ the contrast (or luminosity for the case of nulling). It could also include several companions or other types of detectable features.

The model vector of interest becomes the model signal that maximizes the likelihood of the data, called the Maximum Likelihood Estimate (MLE):
$$ \hat{\mathbf{x}} :=  \underset{\mathbf{a}}{\mathrm{argmax}}\Big( \mathscr{l}(\mathbf{a};\mathbf{y}) \Big).$$
This MLE can not always be computed analytically. In the general case, one will rely on a gradient descent and a good starting point estimate (or tricks like simulated annealing) to identify it reliably.

As derived in **Ceau et al. 2019** in equations (30) and (31), the resulting test can be written:
$$ T_{B}(\mathbf{y}) = 2 \mathbf{y}^T\hat{\mathbf{x}} - \hat{\mathbf{x}}^T\hat{\mathbf{x}} \underset{\mathcal{H}_0}{\overset{\mathcal{H}_1}{\gtrless}} \xi $$

In [ ]:
def T_B(y, xhat):
    Tb = 2*y.T.dot(xhat) - xhat.T.dot(xhat)
    return Tb


As a consequence, the distribution of $T_B(y)$ is not known analytically, even under $\mathcal{H}_0$, and therefore the determination of $P_{FA}$ requires tedious processes, like the Monte Carlo simulations used in the paper. Furthermore, computing $P_{Det}$ to evaluate the performance requires more MC simulations under $\mathcal{H}_1$, and for eacho of the model parameters considered (like a 3D grid for a single companion detection).



## Model fitting

So we will need a reliable way to obtain a MLE. The library `lmfit` is a good choice, as it provides access to a wide array of powerful tools with a nice API. Direct use of the methods like `scipy.optimize.leastsq` is also a popular choice, and is functionnally very similar to what will be shown here.

In [ ]:
from lmfit import minimize, Parameters
parameters = Parameters()
parameters.add("A",value=1.,  min=0, max=4.)
parameters.add("omega", value=0.5, min=0., max=1.)
parameters.add("phi", value=0.2, min=-np.pi, max=np.pi)

In [ ]:
from IPython.display import display
def residual_sine(myparameters, target):
    myA = myparameters["A"]
    myomega = myparameters["omega"]
    myphi = myparameters["phi"]
    asignal = myA*np.sin(myomega*t + myphi)
    return asignal - target
def residual_sine_quick(myparameters, target):
    asignal = myparameters["A"]*np.sin(myparameters["omega"]*t + myparameters["phi"])
    return asignal - target
def get_MLE(parameters, y, verbose=False, method="leastsq", **fit_kws):
    sol = minimize(residual_sine, parameters, args=(y,), method=method, **fit_kws)
    xhat = residual_sine(sol.params, np.zeros_like(y))
    if verbose:
        display(sol)
    return xhat


def get_MLE_robust(parameters, y, verbose=False, method=None, **fit_kws):
    sol0 = minimize(residual_sine_quick, parameters, args=(y,), method="brute", **fit_kws)
    sol = minimize(residual_sine_quick, sol0.params, args=(y,), method="leastsq", **fit_kws)
    xhat = residual_sine(sol.params, np.zeros_like(y))
    if verbose:
        display(sol)
    return xhat

We build a large set of observations to do this MC simulation.

In [ ]:
y_0s = noise_bank[:500,:]
y_1s = x + noise_bank[-500:,:] # We will call the noise bank from the bottom for the signals under H_1

Here is an example of the fitting for the parameters under each of the hypothesis. For the characterization under $\mathcal{H}_0$, it is important to find the global maximum of the likelihood, so we use two steps.

1. A brute-force approach identifies the most promising valley.
2. A gradient descent algorithm finds the corresponding local minimum (or maximum in this case).

Note that in practice one can often leverage the linearity with regard to the contrast parameter to explore lower-dimension **correlation maps** or **colinearity maps** in order to find an acceptable starting point for the gradient descent.

In [ ]:
xhat1 = get_MLE_robust(parameters, y_1s[0], verbose=True)

In [ ]:
xhat0 = get_MLE_robust(parameters, y_0s[0], verbose=True, method="brute")

In [ ]:
fig = plt.figure(dpi=150)
plt.plot(t, y_0, label=f"$\mathbf{{y}}$ (obsered under $\mathcal{{H}}_0$ just noise)",
        color="C0")
plt.plot(t, xhat0, label=f"$\hat{{\mathbf{{x}}}}$ (fitted under $\mathcal{{H}}_0$, just noise)",
        color="C0", linestyle="--")
plt.plot(t, y_1, label=f"$\mathbf{{y}}$ (observed under $\mathcal{{H}}_1$, noisy signal)",
        color="C1")
plt.plot(t, xhat1, label=f"$\hat{{\mathbf{{x}}}}$ (fitted under $\mathcal{{H}}_1$, noisy signal)",
        color="C1", linestyle="--")
plt.plot(t, x, label=f"$\mathbf{{x}}$ (model signal)",
        color="gray", linestyle="--")
plt.legend(fontsize="xx-small")
plt.show()

### Fig. 6: Plot comparing signals and their MLE models (best fit) under the different hypotheses. While in the case of $\mathcal{H}_0$, the fitted signal fits only noise, the fitted signal under $\mathcal{H}_1$ is a good approximation of the model signal (indicating that it has found the correct local optimum). This has the effect of moving the problem to a subspace of fewer dimensions (the model parameters, rather than the number of observables). However, the whole reasonning relies on the validity of this model.

In order to obtain a good evaluation of the performance, a large number of realizations must be used, leading to possibly long execution times.

In [ ]:
from tqdm import tqdm
xhat_0s = []
for i in tqdm(range(y_0s.shape[0])):
    myxhat = get_MLE_robust(parameters, y_0s[i,:])
    xhat_0s.append(myxhat)
xhat_0s = np.array(xhat_0s)

In [ ]:
xhat_1s = []
for i in tqdm(range(y_1s.shape[0])):
    myxhat = get_MLE_robust(parameters, y_1s[i,:])
    xhat_1s.append(myxhat)
xhat_1s = np.array(xhat_1s)

In [ ]:
T_B0s = np.array([T_B(y_0s[i,:], xhat_0s[i,:]) for i in range(y_0s.shape[0])])
T_B1s = np.array([T_B(y_1s[i,:], xhat_1s[i,:]) for i in range(y_1s.shape[0])])

In [ ]:
plt.figure(dpi=150)
plt.hist(T_B0s, histtype="step", bins=50, label=f"($T_{{B}}(\mathbf{{y}}) | \mathcal{{H}}_0$)")
plt.hist(T_B1s, histtype="step", bins=50, label=f"($T_{{B}}(\mathbf{{y}}) | \mathcal{{H}}_1$)")
plt.legend()
plt.xlabel(f"$T_{{B}}$")
plt.ylabel(f"$\mathrm{{hist}}(T_{{B}})$")
plt.show()

### Fig. 7: Histograms of the ditribution of $T_B$ under both hypotheses. This may be seen as an empirical counterpart to Figures 3. and 4., as there are no theoretical distributions for this test.

Based on these, we can compute the ROC for $T_E$ by comparing the number of larger and smaller results under bot $\mathcal{H}_0$ and $\mathcal{H}_1$.

In [ ]:
Tmax = np.max(T_B1s)
Tmin = np.min(T_B0s)
NTs = T_B0s.shape[0]
xsis_B = np.linspace(Tmin, Tmax, 100)
Pfas_B = []
Pdets_B = []
for i, axsi in enumerate(xsis_B):
    Pfas_B.append(1/NTs * np.count_nonzero(T_B0s>axsi))
    Pdets_B.append(1/NTs * np.count_nonzero(T_B1s>axsi))
Pfas_B = np.array(Pfas_B)
Pdets_B = np.array(Pdets_B)

In [ ]:
#Pfas = np.linspace(0., 0.1, 1000)
#Pfas = np.logspace(-10,-0.2, 100)
#Pdet_NP_Pfa = 1 - norm.cdf((np.sqrt(xTx)*norm.ppf(1-Pfas) - xTx)/np.sqrt(xTx))
#Pdet_E_Pfa = 1 - ncx2.cdf(ncx2.ppf(1-Pfas, ndim, 0.), ndim, xTx)
fig = plt.figure(dpi=150)
plt.plot(Pfas, Pdet_NP_Pfa, label=f"ROC($T_{{NP}}$)")
plt.plot(Pfas, Pdet_E_Pfa, label=f"ROC($T_{{E}}$)")
plt.plot(Pfas_B, Pdets_B, label=f"ROC($T_{{B}}$)")
plt.plot(Pfas, Pfas, color="gray", linestyle="--", label="Random")
plt.axvline(Pfa, color="gray", label=f"$P_{{FA}}={Pfa:.3f}$")
#plt.gca().set_aspect("equal")
#plt.xscale("log")
#plt.yscale("log")
plt.xlabel(f"$P_{{FA}}(\\xi)$")
plt.ylabel(f"$P_{{Det}}(\\xi)$")
plt.ylim(0, 1)
plt.xlim(0, np.max(Pfas))
plt.legend()
plt.show()


### Fig. 8: ROC plot comparing the three different tests. The plot for $T_B$ bears some uncertainty as it was built with a limited number of realizations (1000 by default). However it shows significant improvement over the agnostic $T_E$ test.

## $T_B$ in practice

As we can see in this compared ROC plot, $T_B$ is much more sensitive than $T_E$ at the cost of asumptions on the observed signal, but it also provides a MLE of the model parameters as a by-product.

On the other hand, picking a value for a threshold on the basis of a target $P_{FA}$ is non-trivial, and may require some heavy computation cost. In practice a acceptable approach is to ask the question: "If this is a detection ($\xi_{\mathbf{y}} = T_{B}(\mathbf{y})$), then what would be the false alarm rate $\hat{P_{FA}} = P_{FA, \xi_{\mathbf{y}}}$?". The problem is that an accurate determination of this can still take some time. The approach proposed in my PhD thesis (Laugier 2020) is to use an iterative approach to compute with some flexibility an upper bound for this probability.

In this approach, one draws successively MC realizations of $\mathbf{y}$ under $\mathcal{H}_0$ to compare them to $T_{B}(\mathbf{y})$, and each time increments either $n_+$ or $n_-$, dependting if the result was larger or smaller than $T_{B}(\mathbf{y})$. Two conditions can be used to stop:

* The total number of loops exceeds a value $N$ (e.g. $N=100$)
    * In allowing an evaluation of $\hat{P_{FA}}$ down to about $1/N$
* The counter $n_+$ exceeds a value $n$ (e.g. $n=N/10$)
    * Therefore allowing to stop the computation if the False alarm rate is estimated larger than e.g. 10%
    
In most cases under $\mathcal{H}_0$, the loop will stop at around $2n$ realizations, and at N realizations under  $\mathcal{H}_1$. Such a process can even be resumed (for a promising result) at no additional cost if one wants to improve the upper bound set on $P_{FA}$ after stopping at $N$.

Of course, these are only rough estimates, an must be used with care.

### Let us see an example

First initialize the loop.

In [ ]:
N = 100 # Default 100
n = 10 # Default 10


y_ex = y_1s[0] # Here we just pick one realization for the sake of the example from either y_1s or y_0s.
xhat_ex = get_MLE_robust(parameters, y_ex)
T_B_ex = T_B(y_ex, xhat_ex)
nplus = 0
nminus = 0
nstop = False
i = 0

Then the loop can be ran again and again to improve the accuracy of the result.

In [ ]:
while True:
    i += 1
    y0_ex = np.random.multivariate_normal(mean=np.zeros(ndim),cov=np.eye(ndim), size=1).flatten()
    xhat0_ex = get_MLE_robust(parameters, y0_ex)
    T_B0_ex = T_B(y0_ex, xhat0_ex)
    if T_B0_ex>=T_B_ex:
        nplus += 1
    else :
        nminus += 1
    print(f"Loop {i}/{N} : n+={nplus}, n-={nminus}", end="\r")
    if nplus>n:
        est_pfa = nplus/i
        print("\n")
        print(f"P_FA >= {est_pfa:.4f}")
        break
    if i>=N:
        if nplus == 0:
            est_pfa = 1/i
            print("\n")
            print(f"P_FA <= {est_pfa:.4f}")
        else:
            est_pfa = nplus/i
            print("\n")
            print(f"P_FA ~ {est_pfa:.4f}")
            
        break


In [ ]:
# For more precision, we can just increase N, then relauch the loop.
N=300

# Conclusion

$T_{NP}$ and $T_{E}$ can be used to place bounds on the performance of a given approach as they represent two extremes in the usage of *a priori* knowledge on the target. Both these cases offer mathematical approaches to evaluate performance based on a constrain on $P_{FA}$.

$T_B$ is formalization of a GLR test that can be adapted around any parametric model. This flexibility comes at the cost of simplicity, as it becomes difficult to evaluate the performance, and $P_{FA}$. Although an evaluation of the performance on a large set of parameters is computationaly costly, it can still be used on the field thanks to some algorithmic tricks.

Always keep in mind that the performance (and usability) of the tests depend on two main factors:

* Our capability to produce unbiased measurements, and therefore in practice the quality of our calibration process
* Our capability to produce a reliable estimation of the errors

Those two criteria are not independant. Whatever biases are likely to remain in the data should be taken into account in the error estimates.

The main offenders are therefore errors evolving on intermediate timescales: two fast to be correctly calibrated and too slow to be easily measured in an empirical covariance.

What adds up to that is the problem that a cutting-edge survey with unprecedented sensitivity can expect to make discoveries even around calibrators.

### Appendix A on signal whitening
Steps to fulfill the previously mentionned hypothesis will be illustrated in a different tutorial, but here is a short overview:
* The measurement is unbiased (calibration has been applied).
* The measurement vector has been scaled to a standard deviation of 1.
* The measurement vector $\mathbf{y}$ is expressed in a basis where the error components $\boldsymbol{\varepsilon}$ are uncorrelated.

In the case of observable vectors that are normally distributed and centered around 0 under $\mathcal{H}_0$, this can be obtained from the correlated and arbitrarily scaled values through the use of a simple whitening transformation.
$$ \mathbf{y}_p = \mathbf{x}_p + \boldsymbol{\varepsilon}_p$$
$$ \boldsymbol{\varepsilon}_p \sim \mathcal{N}(0, \boldsymbol{\Sigma}_p) $$
One could also say:
$$ \mathrm{Cov}(\boldsymbol{\varepsilon_p}) = \boldsymbol{\Sigma_p}$$
Then we can obtained the whitened signal $\mathbf{y}$ by taking:
$$ \mathbf{y} = \mathbf{W}.\mathbf{y}_p$$
where $\mathbf{W}$ is the inverse matrix square root of this covariance matrix:
$$ \mathbf{W} = \boldsymbol{\Sigma_p}^{-\frac{1}{2}} .$$
One then obtains $\mathrm{Cov}(\boldsymbol{\varepsilon}) = \boldsymbol{\Sigma} = \mathbf{I}$, and allows us tu use $\mathbf{y}$ with the aformentionned assumptions.